# Setup

In [2]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

# check environment
import os
print(f'Conda Environment: ' + os.environ['CONDA_DEFAULT_ENV'])

from platform import python_version
print(f'python version: {python_version()}')

Conda Environment: rich_clust
python version: 3.9.12


In [3]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import natsort

import torch
# from kymatio.torch import Scattering2D

import gc
import time
import functools
import multiprocessing as mp


In [4]:
dir_save = '/home/rich/Desktop/'

In [12]:
params = {
    'paths': {
        'dir_github': r'/media/rich/Home_Linux_partition/github_repos/',  ## directory where ROICat is
        'dir_allOuterFolders': r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp16",  ## directory where directories containing below 'pathSuffixTo...' are
        'pathSuffixToStat': 'plane0/stat.npy',  ## path suffix to where the stat.npy file is
        'pathSuffixToOps': 'plane0/ops.npy',  ## path suffix to where the ops.npy file is
    },
    'importing': {
        'data_verbose': True,  ## default: True. Whether to print out data importing information
        'out_height_width': [72, 72],  ## default: [36,36]. Height and width of small cropped output images of each ROI.
        'max_footprint_width': 1025,  ## default: 1025. Maximum length of a spatial footprint. If you get an error during importing, try increasing this value.
        'type_meanImg': 'meanImgE',  ## default: 'meanImgE'. Type of mean image to use for normalization. This is just a field in the ops.npy file.
        'um_per_pixel': 1.0,  ## default: 1.0. Number of microns per pixel for the imaging dataset. Doesn't need to be exact. Used for resizing the ROIs. Check the images of the resized ROIs to tweak.
        'new_or_old_suite2p': 'new',  ## default: 'new'. If using suite2p, this specifices whether the stat.npy file is in the old MATLAB format or new Python format.
        'images': None,  ## default: None. Set to None if you want to use the images extracted from Suite2p
        'import_workers': -1, ## default: -1. Number of workers to use for importing. Set to -1 to use all available workers.
    },
    'alignment': {
        'session_template': 0.5,  ## default: 0.5. Which session to use as a registration template. If input is float (ie 0.0, 0.5, 1.0, etc.), then it is the fractional position of the session to use; if input is int (ie 1, 2, 3), then it is the index of the session to use (0-indexed)
        'phaseCorr': {
            'freqHighPass': 0.01,  ## default: 0.01. Spatial frequency upper-bound cut-off to use for phase correlation. Spatial frequencies correlations higher than this will be set to zero.
            'freqLowPass': 0.3,  ## default: 0.3. Spatial frequency lower-bound cut-off to use for phase correlation. Spatial frequencies correlations lower than this will be set to zero.
        },
        'method': 'createOptFlow_DeepFlow',  ## default: 'createOptFlow_DeepFlow'. Method to use for creating optical flow.
        'kwargs_method': None,  ## default: None. Keyword arguments to pass to the cv2 optical flow method.
        'use_CLAHE': False,  ## default: False. Whether or not to use 'Contrast Limited Adaptive Histogram Equalization'. Useful if params['importing']['type_meanImg'] is not a contrast enhanced image (like 'meanImgE' in Suite2p)
        'return_sparse': True,  ## default: True. Whether to return a sparse matrix (True) or a dense matrix (False).
        'normalize': True,  ## default: True. If True, normalize the spatial footprints to have a sum of 1.
    },
    'blurring': {
        'kernel_halfWidth': 4.0,  ## default: 2.0. Half-width of the cosine kernel used for blurring. Set value based on how much you think the ROIs move from session to session.
        'plot_kernel': False,  ## default: False. Whether to plot the kernel used for blurring.
    },
    'ROInet': {
        'device': 'cuda:0',  ## default: 'cuda:0'. Device to use for ROInet. Recommend using a GPU.
        'dir_networkFiles': '/home/rich/Downloads/ROInet',  ## local directory where network files are stored
        'download_from_gDrive': 'check_local_first',  ## default: 'check_local_first'. Whether to download the network files from Google Drive or to use the local files.
        'gDriveID': '1D2Qa-YUNX176Q-wgboGflW0K6un7KYeN',  ## default: '1FCcPZUuOR7xG-hdO6Ei6mx8YnKysVsa8'. Google Drive ID of the network files.
        'forward_pass_version':  # default: 'latent'. Leave as 'latent' for most things. Can be 'latent' (full pass through network), 'head' (output of the head layers), or 'base' (pass through just base layers)
        'verbose': True,  ## default: True. Whether to print out ROInet information.
        'pref_plot': False,  ## default: False. Whether to plot the ROI and the normalized ROI.
        'batchSize_dataloader': 8,  ## default: 8. Number of images to use for each batch.
        'pinMemory_dataloader': True,  ## default: True. Whether to pin the memory of the dataloader.
        'persistentWorkers_dataloader': True,  ## default: True. Whether to use persistent workers for the dataloader.
        'prefetchFactor_dataloader': 2,  ## default: 2. Number of prefetch factors to use for the dataloader.
    },
    'SWT': {
        'kwargs_Scattering2D': {'J': 2, 'L': 2},  ## default: {'J': 2, 'L': 8}. Keyword arguments to pass to the Scattering2D function.
        'device': 'cuda:0',  ## default: 'cuda:0'. Device to use for SWT. Recommend using a GPU.
    }, 
    'similarity': {
        'spatialFootprint_maskPower': 1.0,  ## default: 1.0. This determines the power to take the ROI mask images to. Higher for more dependent on brightness, lower for more dependent on binary overlap.
        'n_workers': -1,  ## default: -1. Number of workers to use for similarity. Set to -1 to use all available workers.
        'block_height': 512,  ## default: 512. Maximum height of the FOV block bins to use for pairwise ROI similarity calculations.
        'block_width': 512,  ## default: 512. Maximum width of the FOV block bins to use for pairwise ROI similarity calculations.
        'algorithm_nearestNeigbors_spatialFootprints': 'brute',  ## default: 'brute'. Algorithm to use for nearest neighbors.
        'verbose': True,  ## default: True. Whether to print out similarity information.
        'normalization': {
            'k_max': 4000,  ## default: 4000. Maximum kNN distance to use for building a distribution of pairwise similarities for each ROI.
            'k_min': 150,  ## default: 150. Set around n_sessions*10. Minimum kNN distance to use for building a distribution of pairwise similarities for each ROI. 
            'algo_NN': 'kd_tree',
        },
    },
    ## Cluster
clusterer = clustering.Clusterer()

clusterer.make_conjunctive_distance_matrix(
    s_sf=sim.s_sf,
    s_NN_z=sim.s_NN_z,
    s_SWT_z=sim.s_SWT_z,
    power_sf=params['clustering']['power_sf'],
    power_NN=params['clustering']['power_NN'],
    power_SWT=params['clustering']['power_SWT'],
    p_norm=params['clustering']['p_norm'],
    sig_NN_kwargs=params['clustering']['sig_NN_kwargs'],
    sig_SWT_kwargs=params['clustering']['sig_SWT_kwargs'],
    plot_sigmoid=params['clustering']['plot_pref'],
)

# %matplotlib inline
clusterer.plot_similarity_relationships(plots_to_show=[1,2,3])

clusterer.find_intermode_cutoff(
    n_bins=params['clustering']['n_bins'],
    smoothing_window=params['clustering']['smoothing_window'],
    plot_pref=params['clustering']['plot_pref'],
)

labels = clusterer.fit(
    session_bool=data.sessionID_concat,
    min_cluster_size=params['clustering']['min_cluster_size'],
    discard_failed_pruning=params['clustering']['discard_failed_pruning'],
);

    'clustering': {
        'power_sf': 1,  ## default: 1. Power to which to raise the spatial footprint similarity. Tune by looking at the clusterer.plot_similarity_relationships() plots.
        'power_NN': 2,  ## default: 2. Power to which to raise the spatial neural network similarity. Tune by looking at the clusterer.plot_similarity_relationships() plots.
        'power_SWT': 0.1,  ## default: 0.1. Power to which to raise the scattering wavelet transform similarity. Tune by looking at the clusterer.plot_similarity_relationships() plots.
        'p_norm': -1,  ## default: -1. p-norm to use for the conjunction of the similarity matrices into a single matrix. Higher values are more conjunctive (more like s_sf * s_NN), and lower values are more permissive (more like s_sf + s_NN)
        'sig_NN_kwargs':{'mu':0.0, 'b':0.5},
        'sig_SWT_kwargs':{'mu':0.0, 'b':0.5},
        'plot_pref':True,
    },
    'visualization': {
        'FOV_threshold_confidence': 0.5,  ## default: 0.5. Threshold for the confidence scores when displaying ROIs.
    },
}

In [11]:
dir_github = Path(params['paths']['dir_github']).resolve()

import sys
sys.path.append(str(dir_github))

%load_ext autoreload
%autoreload 2
from ROICaT.tracking import data_importing, visualization, alignment, blurring, helpers, ROInet, scatteringWaveletTransformer, similarity_graph, clustering

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# params_all = [helpers.deep_update_dict(params, ['paths', 'dir_allOuterFolders'], val) for val in [
#         r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp6_3",
#         r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp10",
#         r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp11",
#         r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp12",
#         r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp13",
#         r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp16",
# ]]

In [15]:
type(1) is float

False

In [ ]:
# %matplotlib notebook


# Import paths
def print_list(l):
    for item in l:
        print(item)

dir_allOuterFolders = Path(params['paths']['dir_allOuterFolders']).resolve()

folders_allSessions = natsort.natsorted(helpers.get_dir_contents(dir_allOuterFolders)[0])

folders_toUse = folders_allSessions
# folders_toUse = list(map(folders_allSessions.__getitem__, [np.arange(1,9, dtype=np.int32)]))


# dir_allS2pFolders = [dir_allOuterFolders / folder / 'exp' / 'suite2p' / 'plane0' for folder in folders_toUse]
dir_allS2pFolders = [dir_allOuterFolders / folder for folder in folders_toUse]

pathSuffixToStat = params['paths']['pathSuffixToStat']
pathSuffixToOps = params['paths']['pathSuffixToOps']

paths_allStat = np.array([path / pathSuffixToStat for path in dir_allS2pFolders])[:2]
paths_allOps  = np.array([path / pathSuffixToOps for path in dir_allS2pFolders])[:2]

print(folders_allSessions)
print(folders_toUse)
print_list(dir_allS2pFolders)
print_list(paths_allStat)


#Import data
data = data_importing.Data_suite2p(
    paths_statFiles=paths_allStat,
    paths_opsFiles=paths_allOps,
    um_per_pixel=params['importing']['um_per_pixel'],    
    new_or_old_suite2p=params['importing']['new_or_old_suite2p'],
    verbose=params['importing']['data_verbose'],
);

data.import_statFiles();

data.import_ROI_centeredImages(
    out_height_width=params['importing']['out_height_width'],
    max_footprint_width=params['importing']['max_footprint_width'],
);

data.import_FOV_images(
    type_meanImg=params['importing']['type_meanImg'],
    images=params['importing']['images'],
);

data.import_ROI_spatialFootprints(workers=params['importing']['import_workers']);

# visualization.display_toggle_image_stack(data.FOV_images)


# Alignment
pcr = alignment.PhaseCorrelation_registration()

pcr.set_spectral_mask(
    freq_highPass=params['alignment']['phaseCorr']['freq_highPass'],
    freq_lowPass=params['alignment']['phaseCorr']['freq_lowhPass'],
    im_shape=(data.FOV_height, data.FOV_width)
)

st = params['alignment']['session_template']
idx_st = int(st * data.n_sessions) if type(st) is float else st
pcr.register(
    im_template=data.FOV_images[idx_st],
    ims_moving=data.FOV_images
);

FOV_images = [alignment.clahe(im, grid_size=10, clipLimit=0, normalize=True) for im in pcr.ims_registered[:]] if params['alignment']['use_CLAHE'] else data.FOV_images

aligner = alignment.Alinger(
    method=params['alignment']['method'],
    kwargs_method=params['alignment']['kwargs_method'],
)

aligner.register_ROIs(
    templateFOV=FOV_images[idx_st],
    FOVs=FOV_images,
    ROIs=data.spatialFootprints,
    return_sparse=params['alignment']['return_sparse'],
    normalize=params['alignment']['normalize'],
);

# visualization.display_toggle_image_stack(aligner.FOVs_aligned)
# visualization.display_toggle_image_stack(aligner.get_ROIsAligned_maxIntensityProjection())


# Blur ROIs (optional)
blurrer = blurring.ROI_Blurrer(
    frame_shape=(data.FOV_height, data.FOV_width),
    kernel_halfWidth=params['blurring']['kernel_halfWidth'],
    plot_kernel=params['blurring']['plot_kernel'],
)

blurrer.blur_ROIs(
    spatialFootprints=aligner.ROIs_aligned,
)

# visualization.display_toggle_image_stack(blurrer.get_ROIsBlurred_maxIntensityProjection())


# Neural network embedding distances

hash_dict_true = {
    'params': ('params.json', '877e17df8fa511a03bc99cd507a54403'),
    'model': ('model.py', '6ef5c29793ae16a64e43e8cab33d9ff4'),
    'state_dict': ('ConvNext_tiny__1_0_unfrozen__simCLR.pth', 'a5fae4c9ea95f2c78b4690222b2928a5'),
}

roinet = ROInet.ROInet_embedder(
    device=params['ROInet']['device'],
    dir_networkFiles=params['ROInet']['dir_networkFiles'],
    download_from_gDrive=params['ROInet']['download_from_gDrive'],
    gDriveID=params['ROInet']['gDriveID'],
    hash_dict_networkFiles=hash_dict_true,
    verbose=params['ROInet']['verbose'],
)

roinet.generate_dataloader(
    ROI_images=data.ROI_images,
    um_per_pixel=params['importing']['um_per_pixel'],
    pref_plot=params['ROInet']['pref_plot'],
    batchSize_dataloader=params['ROInet']['batchSize_dataloader'],
    pinMemory_dataloader=params['ROInet']['pinMemory_dataloader'],
    numWorkers_dataloader=mp.cpu_count(),
    persistentWorkers_dataloader=params['ROInet']['persistentWorkers_dataloader'],
    prefetchFactor_dataloader=params['ROInet']['prefetchFactor_dataloader'],    
);

# visualization.display_toggle_image_stack(roinet.ROI_images_rs)

roinet.generate_latents();

gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()


# Scattering wavelet embedding distances
swt = scatteringWaveletTransformer.SWT(
    kwargs_Scattering2D=params['SWT']['kwargs_Scattering2D'], 
    image_shape=params['importing']['out_height_width'], 
    device=params['SWT']['device'],
)

swt.transform(ROI_images=np.concatenate(data.ROI_images, axis=0));

# Compute similarities
sim = similarity_graph.ROI_graph(
    n_workers=params['similarity']['n_workers'],
    frame_height=data.FOV_height,
    frame_width=data.FOV_width,
    block_height=params['similarity']['block_height'],
    block_width=params['similarity']['block_width'],
    algorithm_nearestNeigbors_spatialFootprints=params['similarity']['algorithm_nearestNeigbors_spatialFootprints'],
    verbose=params['similarity']['verbose'],
)

sim.visualize_blocks()

sim.compute_similarity_blockwise(
    spatialFootprints=blurrer.ROIs_blurred,
    features_NN=roinet.latents,
    features_SWT=swt.latents,
    ROI_session_bool=data.sessionID_concat,
    spatialFootprint_maskPower=params['similarity']['spatialFootprint_maskPower'],
);

sim.make_normalized_similarities(
    centers_of_mass=data.get_midCoords(),
    features_NN=roinet.latents,
    features_SWT=swt.latents,
    k_max=params['similarity']['normalization']['k_max'],
    k_min=params['similarity']['normalization']['k_min'],
    algo_NN=params['similarity']['normalization']['algo_NN'],
)


## Cluster
clusterer = clustering.Clusterer()

clusterer.make_conjunctive_distance_matrix(
    s_sf=sim.s_sf,
    s_NN_z=sim.s_NN_z,
    s_SWT_z=sim.s_SWT_z,
    power_sf=params['clustering']['power_sf'],
    power_NN=params['clustering']['power_NN'],
    power_SWT=params['clustering']['power_SWT'],
    p_norm=params['clustering']['p_norm'],
    sig_NN_kwargs=params['clustering']['sig_NN_kwargs'],
    sig_SWT_kwargs=params['clustering']['sig_SWT_kwargs'],
    plot_sigmoid=params['clustering']['plot_pref'],
)

# %matplotlib inline
clusterer.plot_similarity_relationships(plots_to_show=[1,2,3])

clusterer.find_intermode_cutoff(
    n_bins=params['clustering']['n_bins'],
    smoothing_window=params['clustering']['smoothing_window'],
    plot_pref=params['clustering']['plot_pref'],
)

labels = clusterer.fit(
    session_bool=data.sessionID_concat,
    min_cluster_size=params['clustering']['min_cluster_size'],
    discard_failed_pruning=params['clustering']['discard_failed_pruning'],
);

# visualization
FOV_clusters = visualization.compute_colored_FOV(
    spatialFootprints=aligner.ROIs_aligned,
    FOV_height=data.FOV_height,
    FOV_width=data.FOV_width,
    preds=ca.preds,
    confidence=ca.confidence,
    threshold_confidence = params['visualization']['FOV_threshold_confidence'],
#     threshold_confidence = 0.99,
)

# %matplotlib notebook
# visualization.display_toggle_image_stack(FOV_clusters)

preds_by_session = [preds[idx].numpy() for idx in data.sessionID_concat.T]

ROIs = {
    "ROIs_aligned": aligner.ROIs_aligned,
    "ROIs_raw": data.spatialFootprints,
    "frame_height": data.FOV_height,
    "frame_width": data.FOV_width,
    "idx_roi_session": [np.where(idx)[0] for idx in data.sessionID_concat.T]
}

name_save = os.path.split(dir_allOuterFolders)[-1]
helpers.simple_save(
    {
        "UCIDs": list(ca.preds.numpy().astype(np.int64)),
        "UCIDs_bySession": preds_by_session,
        "ROIs": ROIs,
    },
    filename=Path(dir_save) / (name_save + '.plane0.rClust' '.pkl'),
#     filename='/media/rich/bigSSD/analysis_data/mouse 2_6/multiday_alignment/UCIDs.pkl'
)

gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# # %matplotlib notebook
# visualization.display_toggle_image_stack(FOV_clusters)


In [ ]:
# import numpy as np
# import scipy.sparse

# import torch_sparse as ts

# scipy.sparse.save_npz(
#     file=r'/home/rich/Desktop/c_sim.npz',
#     matrix=sim.c_sim.tocsr(),
#     compressed=True
# )
# scipy.sparse.save_npz(
#     file=r'/home/rich/Desktop/cluster_bool.npz',
#     matrix=sim.cluster_bool.tocsr(),
#     compressed=True
# )
# np.save(
#     file=r'/home/rich/Desktop/scores.npy',
#     arr=sim.scores.numpy(),
# )

# c_sim = scipy.sparse.load_npz(file=r'/home/rich/Desktop/c_sim.npz').tolil()
# cluster_bool = scipy.sparse.load_npz(file=r'/home/rich/Desktop/cluster_bool.npz').tocsr()
# scores = torch.as_tensor(np.load(file=r'/home/rich/Desktop/scores.npy'), dtype=torch.float32)